<a href="https://colab.research.google.com/github/srini229/EE5333_tutorials/blob/master/drc/Design_Rule_Check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --break-system-packages --force https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/parser/LEFDEFParser-0.1-cp310-cp310-linux_x86_64.whl

In [ ]:
!wget https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/parser/Nangate.lef
!wget https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/drc/rects.txt

In [ ]:
# find parallel run length between two rectangles
# smaller dimension is assumed to be the width
def getPRL(r1, r2):
  w1, h1 = r1.ur.x - r1.ll.x, r1.ur.y - r1.ll.y
  w2, h2 = r2.ur.x - r2.ll.x, r2.ur.y - r2.ll.y
  if w1 > h1 and w2 > h2:
    if r1.ur.x > r2.ll.x and r2.ur.x > r1.ll.x:
      return min(r1.ur.x - r2.ll.x, r2.ur.x - r1.ll.x)
    return 0
  if r1.ur.y > r2.ll.y and r2.ur.y > r1.ll.y:
    return min(r1.ur.y - r2.ll.y, r2.ur.y - r1.ll.y)
  return 0

In [ ]:
# get the spacing value from the spacing table for two input rectangles
def getSpacing(layer, r1, r2):
  width1, width2, prl =   min(r1.ur.x - r1.ll.x, r1.ur.y - r1.ll.y), min(r2.ur.x - r2.ll.x, r2.ur.y - r2.ll.y), getPRL(r1, r2)
  spacing = layer.spacing()
  i = 0
  for i, w in enumerate(layer.spacingTable().width):
    if width1 < w:
      break
  ind1, i = i - 1, 0
  for i, w in enumerate(layer.spacingTable().width):
    if width2 < w:
      break
  ind2, i = i - 1, 0
  for i, p in enumerate(layer.spacingTable().prl):
    if prl < p:
      break
  indp = i - 1
  if ind1 >= 0 and ind2 >= 0 and indp >= 0:
    spacing = max(layer.spacingTable().spacing[ind1][indp], layer.spacingTable().spacing[ind2][indp], spacing)
  return spacing

In [ ]:
# check if two rectangles violate spacing
def checkSpacing(r1, r2, s):
  r = LEFDEFParser.Rect(r1.ll.x - s, r1.ll.y - s, r1.ur.x + s, r1.ur.y + s) # bloat r1 by s on all sides
  return (r.ur.x > r2.ll.x and r.ll.x < r2.ur.x and r.ur.y > r2.ll.y and r.ll.y < r2.ur.y)

In [ ]:
import LEFDEFParser
l = LEFDEFParser.LEFReader()
l.readLEF('Nangate.lef')
#for m in l.macros():
#    m.print()
layerMap = {layer.name():layer for layer in l.layers()}

'''
for k,v in layerMap.items():
  print(v.name(), v.width(), v.spacing())
  print('prl-> ', '\t\t', ' '.join([str(i) for i in v.spacingTable().prl]))
  for i, w in enumerate(v.spacingTable().width):
    #print('width ', v.spacingTable().width[i], '\t', ' '.join([str(i) for i in v.spacingTable().spacing[i]]))
'''

maxSpacing = {layer.name():max(layer.spacing(), max([max(s) for s in layer.spacingTable().spacing])  if len(layer.spacingTable().spacing) else 0) for layer in l.layers()}
print(maxSpacing)


{'metal1': 130, 'metal2': 3000, 'metal3': 3000, 'metal4': 3000, 'metal5': 3000, 'metal6': 3000, 'metal7': 3000, 'metal8': 3000, 'metal9': 3000, 'metal10': 3000}


In [ ]:
layer = 'metal2'
rects = [LEFDEFParser.Rect(0, 0, 1000, 140), LEFDEFParser.Rect(400, 300, 1000, 440), LEFDEFParser.Rect(1100, 0, 1600, 140)]
for i in range(len(rects)):
  for j in range(i + 1, len(rects)):
    print(rects[i], rects[j], checkSpacing(rects[i], rects[j], getSpacing(layerMap[layer], rects[i], rects[j])))


[(0,0),(1000,140)] [(400,300),(1000,440)] False
[(0,0),(1000,140)] [(1100,0),(1600,140)] True
[(400,300),(1000,440)] [(1100,0),(1600,140)] False


In [ ]:
with open('rects.txt', 'r') as fh:
  rects = []
  for l in fh:
    s = [int(i) for i in l.split()]
    if len(s) == 4:
      rects.append(LEFDEFParser.Rect(s[0], s[1], s[2], s[3]))
import time
for N in [100, 200, 500, 1000]:
  for layer in ['metal1', 'metal2']:
    count, t = 0, time.time()
# brute-force quadratic check between every pair of rectangles
    for i in range(min(N, len(rects))):
      for j in range(i + 1, min(N, len(rects))):
        if checkSpacing(rects[i], rects[j], getSpacing(layerMap[layer], rects[i], rects[j])): count += 1
    print('layer :', layer, '#violations :', count, 'runtime :', time.time() - t)
  print()


layer : metal1 #violations : 144 runtime : 0.24188518524169922
layer : metal2 #violations : 165 runtime : 0.2995140552520752

layer : metal1 #violations : 626 runtime : 1.0119891166687012
layer : metal2 #violations : 716 runtime : 1.1504695415496826

layer : metal1 #violations : 3931 runtime : 7.467454433441162
layer : metal2 #violations : 4478 runtime : 8.31714153289795

layer : metal1 #violations : 16165 runtime : 28.18099308013916
layer : metal2 #violations : 18508 runtime : 31.86258363723755



In [ ]:
# build a kdtree of rectangles
# build the kd-tree using the input set of rectangles
class kdtree:
  def __init__(self, rects):
    self._rects = rects
  def region_query(self, region):
    l = []
    return l

In [ ]:
kdt = kdtree(rects)
for N in [100, 1000, 2000]:
  t = time.time()
  for layer in ['metal1', 'metal2', 'metal3']:
    count = 0
    for i in range(min(N, len(rects))):
      # replace pair-wise check with potentially conflicting neighbours through region query
      s = maxSpacing[layer]
      nbrs = kdt.region_query(LEFDEFParser.Rect(rects[i].ll.x - s, rects[i].ll.y - s, rects[i].ur.x + s, rects[i].ur.y + s))
      for j in range(len(nbrs)):
        if rects[i] == nbrs[j]: continue
        if checkSpacing(rects[i], rects[j], getSpacing(layerMap[layer], rects[i], nbrs[j])): count += 1
    print(layer, count)
  print('runtime : ', time.time() - t)